# Classe RSA

Foi implementada uma classe que permite a geração de equemas RSA. Esta recebe necessariamente um valor como argumento, responsável por determinar o parâmetro de segurança. Com este, efetua-se a arit

In [1]:
from sage.cpython.string import bytes_to_str, str_to_bytes
import hashlib

class RSA:
    def __init__(self, size):

        p = random_prime(2**size-1,True,2**(size-1))
        q = random_prime(2**size-1,True,2**(size-1))
        while p == q:
            p = random_prime(2**size-1,True,2**(size-1))
            q = random_prime(2**size-1,True,2**(size-1))
        
        self.N = p*q
        phi = (p - 1) * (q - 1)
        
        G = IntegerModRing(phi)
        self.pub = G.random_element(phi)
        while gcd(self.pub, phi) != 1:
            self.pub = G.random_element(phi)
        
        self.priv = 1/self.pub
        
        
    def encrypt(self, msg):
        pub = self.pub
        G = IntegerModRing(self.N)
        ptxt = []
        
        for i in str.upper(msg):
            x = ord(i)
            ptxt.append(x)
            
        ptxtval = int(''.join(map(str, ptxt)))
        
        ctxt = G(ptxtval)**pub
        
        return ctxt
    
    
    def decrypt(self, crypto):
        priv = self.priv
        msg = ""
        
        ptxt = crypto**priv
        aux = str(ptxt)
        
        for i in xrange(0,len(aux)-1,2):
            x = chr(int(aux[i] + aux[i+1]))
            msg += x
        
        return str.lower(msg)
    
    
    def sign(self, msg):
        G = IntegerModRing(self.N)
        
        hashval = int(hashlib.sha256(str(msg)).hexdigest(), 16)
        
        signature = G(hashval)**self.priv
        
        return signature
    
    
    def verify(self, crypto, sign):
        
        hashval = int(hashlib.sha256(str(crypto)).hexdigest(), 16)
        
        signature = sign**self.pub
        
        if hashval == signature:
            print 'Valid signature'
        else:
            print 'Invalid signature'

# SETUP

In [2]:
# Initialise class and generate key pair
rsa = RSA(1024)

# Message
msg = "exemplo de uma mensagem!"

# Encrypt
ctxt = rsa.encrypt(msg)

print("ctxt = " + str(ctxt))

# Decrypt
ptxt = rsa.decrypt(ctxt)

print("\nptxt = " + ptxt)

# Sign
sign = rsa.sign(ctxt)

print("\nsign = " + str(sign) + "\n")

# Verify
rsa.verify(ctxt, sign)

ctxt = 5648755054659850258521323998661187630384445423406873283309245101886603830916772323617419932930463336449836733034787053103673564964264042039492529499551359497109064336501581161987644379179072571775584826836278318262309425580147186089489717108209969091716170892678424442968042572900808617713093917580785457966481264647548099640271947921088764076083147663666210372249292507643353175503423630077337577506166210595270739383676723170120709817742761458611284910103599095947540358708731714729046038619251960075656625838042131800151506296600165255006806067840847964153727810053473935159822881106223754049943567020571172816412

ptxt = exemplo de uma mensagem!

sign = 65180744071294167722771830919525075531747848441110788118681020414353902072512303160682764658752035985717803963825307997089983250686274924187494451869551005400683578973483697791538690859900876969699627929099581207587418737208618755098301990448227604744221686724241305986140647896987920603163481750821322972422118497123396977554031377610